# Data Reduction Pipeline

This notebook demonstrates the basic CCD data reduction workflow:
1. Create master calibration frames (bias, flat)
2. Apply calibrations to science frames
3. Inspect results

## Prerequisites
- Raw FITS files in `../data/raw/`
- Calibration frames in `../data/calibration/`

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.data_reduction import (
    load_fits,
    create_master_bias,
    create_master_flat,
    reduce_science_frame,
    batch_reduce
)
from src.utils import list_fits_files, plot_image

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Define File Paths

In [ ]:
# Directory paths
data_dir = Path('../data')
raw_dir = data_dir / 'raw'
cal_dir = data_dir / 'calibration'
reduced_dir = data_dir / 'reduced'

# Create output directory if it doesn't exist
reduced_dir.mkdir(exist_ok=True)

print(f"Raw data: {raw_dir}")
print(f"Calibration: {cal_dir}")
print(f"Reduced data: {reduced_dir}")

## 2. Create Master Bias Frame

Bias frames capture the electronic offset of the CCD. We combine multiple bias frames using the median to reduce noise.

In [ ]:
# List all bias frames
bias_files = list_fits_files(cal_dir, pattern='bias*.fits')
print(f"Found {len(bias_files)} bias frames")

# Create master bias if bias files exist
if len(bias_files) > 0:
    master_bias = create_master_bias(
        bias_files,
        output_path=cal_dir / 'master_bias.fits'
    )
    
    # Display master bias
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    plot_image(master_bias, title='Master Bias', ax=axes[0])
    axes[1].hist(master_bias.flatten(), bins=100, alpha=0.7)
    axes[1].set_xlabel('Counts')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Bias Level Distribution')
    plt.tight_layout()
    plt.show()
    
    print(f"Master bias statistics:")
    print(f"  Mean: {np.mean(master_bias):.2f}")
    print(f"  Median: {np.median(master_bias):.2f}")
    print(f"  Std: {np.std(master_bias):.2f}")
else:
    master_bias = None
    print("No bias frames found. Proceeding without bias correction.")

## 3. Create Master Flat Frame

Flat fields correct for pixel-to-pixel sensitivity variations and vignetting.

In [ ]:
# List all flat frames (adjust pattern for your filter)
flat_files = list_fits_files(cal_dir, pattern='flat*.fits')
print(f"Found {len(flat_files)} flat frames")

# Create master flat if flat files exist
if len(flat_files) > 0:
    master_flat = create_master_flat(
        flat_files,
        master_bias=master_bias,
        output_path=cal_dir / 'master_flat.fits'
    )
    
    # Display master flat
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    plot_image(master_flat, title='Master Flat', ax=axes[0], vmin=0.8, vmax=1.2)
    axes[1].hist(master_flat.flatten(), bins=100, alpha=0.7)
    axes[1].set_xlabel('Normalized Response')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Flat Field Distribution')
    axes[1].axvline(1.0, color='r', linestyle='--', label='Ideal')
    axes[1].legend()
    plt.tight_layout()
    plt.show()
    
    print(f"Master flat statistics:")
    print(f"  Mean: {np.mean(master_flat):.4f}")
    print(f"  Median: {np.median(master_flat):.4f}")
    print(f"  Std: {np.std(master_flat):.4f}")
else:
    master_flat = None
    print("No flat frames found. Proceeding without flat fielding.")

## 4. Reduce Science Frames

Apply bias correction and flat fielding to all science frames.

In [ ]:
# List science frames
science_files = list_fits_files(raw_dir)
print(f"Found {len(science_files)} science frames")

# Batch reduce if files exist
if len(science_files) > 0:
    reduced_files = batch_reduce(
        science_files,
        master_bias=master_bias,
        master_flat=master_flat,
        output_dir=str(reduced_dir)
    )
    print(f"Reduced {len(reduced_files)} frames")
else:
    print("No science frames found in raw directory.")

## 5. Compare Raw vs Reduced

Visual comparison of a raw and reduced frame.

In [ ]:
# Compare first frame if available
if len(science_files) > 0 and len(reduced_files) > 0:
    raw_data, _ = load_fits(science_files[0])
    reduced_data, _ = load_fits(reduced_files[0])
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    plot_image(raw_data, title='Raw Frame', ax=axes[0])
    plot_image(reduced_data, title='Reduced Frame', ax=axes[1])
    plt.tight_layout()
    plt.show()
    
    print(f"Background levels:")
    print(f"  Raw: {np.median(raw_data):.2f}")
    print(f"  Reduced: {np.median(reduced_data):.2f}")

## Summary

✓ Master calibration frames created
✓ Science frames reduced
✓ Output saved to `../data/reduced/`

**Next Steps:**
- Proceed to `02_light_curve_analysis.ipynb` for photometry
- Check image quality and star detection
- Adjust parameters if needed